In [8]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import os
import autogluon
from autogluon.tabular import TabularDataset,TabularPredictor
from sklearn.model_selection import train_test_split
import utils

In [ ]:
drop_cols=["nl","el",]

In [2]:
import geopandas as gpd

In [ ]:
gdf_json = gpd.read_file(os.path.join('..', 'data', 'train', 'L02-23.geojson'))[["L02_001","L02_006","L02_024","geometry"]]
print(gdf_json.head())

  L02_001 L02_002 L02_003 L02_004  L02_005  L02_006  L02_007  L02_008 L02_009  \
0     000     001     000     001     2023   245000      4.3        1   false   
1     000     002     000     002     2023   224000     12.0        1   false   
2     000     003     000     003     2023   195000     14.7        1   false   
3     000     004     000     004     2023    65000     14.0        1   false   
4     000     005     000     005     2023   367000      3.4        1   false   

  L02_010  ...         L02_127         L02_128         L02_129  \
0   false  ...  10000000000000  10000000000000  10000000000000   
1   false  ...  00000000000000  40000000000000  10000000000000   
2   false  ...  10000000000000  10000000000000  10000000000000   
3   false  ...  10000000000000  10000000000000  10000000000000   
4   false  ...  10000000000000  10000010000000  10000000000000   

          L02_130         L02_131         L02_132       L02_133       L02_134  \
0  10000000000000  10000000000000  

In [4]:
train_df = pd.read_csv(os.path.join('..', 'data', 'train', 'train.csv'))
test_df = pd.read_csv(os.path.join('..', 'data', 'test', 'test.csv'))

C:\Users\km20186\AppData\Local\Temp\ipykernel_31200\3353662060.py:1: DtypeWarning: Columns (63) have mixed types. Specify dtype option on import or set low_memory=False.
  train_df = pd.read_csv(os.path.join('..', 'data', 'train', 'train.csv'))
C:\Users\km20186\AppData\Local\Temp\ipykernel_31200\3353662060.py:2: DtypeWarning: Columns (46,55,56,63,146) have mixed types. Specify dtype option on import or set low_memory=False.
  test_df = pd.read_csv(os.path.join('..', 'data', 'test', 'test.csv'))


In [ ]:
train_data: pd.DataFrame = TabularDataset(os.path.join('..', 'data', 'train', 'train.csv'))
test_data: pd.DataFrame = TabularDataset(os.path.join('..', 'data', 'test', 'test.csv'))

In [ ]:
train, test = train_test_split(train_df, test_size=0.2, random_state=42)
train_data: pd.DataFrame = TabularDataset(train)
test_data: pd.DataFrame = TabularDataset(test)
model=TabularPredictor(
    label='money_room',
    problem_type='regression',
    eval_metric='mape'
)
predictor=model.fit(
    train_data=train_data,
    test_data=test_data,
    presets='best_quality',
    time_limit=3600,
)

In [ ]:
predictor.leaderboard(test)

In [ ]:
predictions = predictor.predict(test_data)
#

In [ ]:
test_data: pd.DataFrame = TabularDataset(test_data)

In [ ]:
pred=pd.DataFrame(predictions)
pred.head()
pred.to_csv(os.path.join('..', 'data', 'submit', 'submission.csv'), index_label='id')

In [ ]:
train_data.head()

In [ ]:
predictor.feature_importance(test,
                             time_limit=3600
                            )